In [2]:
import pandas as pd
import os

In [6]:
folder_path = '/Users/judith.rethmann/Documents/MBS/PredictiveAnalytics/FlightData'

dataframes = []

for filename in os.listdir(folder_path):
    if filename.startswith('T_ONTIME_REPORTING') and filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)
        dataframes.append(df)

common_columns = set(dataframes[0].columns)
for df in dataframes[1:]:
    common_columns.intersection_update(df.columns)

common_columns = list(common_columns)
df_flight_data = pd.concat([df[common_columns] for df in dataframes], ignore_index=True)

In [7]:
df_flight_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5762468 entries, 0 to 5762467
Data columns (total 15 columns):
 #   Column             Dtype  
---  ------             -----  
 0   DEP_DEL15          float64
 1   DAY_OF_WEEK        int64  
 2   FL_DATE            object 
 3   OP_CARRIER_FL_NUM  int64  
 4   ORIGIN_CITY_NAME   object 
 5   DEST_AIRPORT_ID    int64  
 6   MONTH              int64  
 7   DAY_OF_MONTH       int64  
 8   DEST_CITY_NAME     object 
 9   ORIGIN_AIRPORT_ID  int64  
 10  DEP_TIME           float64
 11  DEP_DELAY          float64
 12  YEAR               int64  
 13  OP_UNIQUE_CARRIER  object 
 14  AIR_TIME           float64
dtypes: float64(4), int64(7), object(4)
memory usage: 659.5+ MB


In [4]:
df_flights_SFO_UA = df_flight_data[(df_flight_data['ORIGIN_CITY_NAME'] == 'San Francisco, CA') & (df_flight_data['OP_UNIQUE_CARRIER'] == 'UA') & (df_flight_data['CANCELLED'] == 0) & (df_flight_data['DIVERTED'] == 0)]


KeyError: 'CANCELLED'

In [ ]:
df_flights_SFO_UA.info()

In [ ]:
'''columns_to_keep = [
    'ORIGIN_CITY_NAME', 'ORIGIN_AIRPORT_ID', 'OP_UNIQUE_CARRIER', 'OP_CARRIER_FL_NUM',
    'FL_DATE', 'YEAR', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'DEST_CITY_NAME',
    'DEST_AIRPORT_ID', 'AIR_TIME', 'DEP_TIME', 'DIVERTED', 'CANCELLED', 'DEP_DELAY',
    'DEP_DEL15', 'NAS_DELAY', 'CARRIER_DELAY', 'SECURITY_DELAY', 'WEATHER_DELAY'
]'''

columns_to_keep = [
    'ORIGIN_CITY_NAME', 'ORIGIN_AIRPORT_ID', 'OP_UNIQUE_CARRIER', 'OP_CARRIER_FL_NUM',
    'FL_DATE', 'YEAR', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'DEST_CITY_NAME',
    'DEST_AIRPORT_ID', 'AIR_TIME', 'DEP_TIME', 'DEP_DELAY', 'DEP_DEL15'
]

df_flights_SFO_UA = df_flights_SFO_UA[columns_to_keep]

In [ ]:
df_flights_SFO_UA.to_csv('/Users/judith.rethmann/Documents/MBS/PredictiveAnalytics/FlightData/flights_SFO_UA.csv', index=False)